In [ ]:
import os
import json
import codecs
from collections import defaultdict
from datetime import datetime

def safe_read_json(file_path):
    try:
        with codecs.open(file_path, 'r', 'utf-8-sig') as f:
            return json.load(f)
    except UnicodeDecodeError:
        with codecs.open(file_path, 'r', 'iso-8859-1') as f:
            return json.load(f)

def count_messages_in_file(file_path):
    try:
        messages = safe_read_json(file_path)
        year_counts = defaultdict(int)
        day_counts = defaultdict(int)
        
        if isinstance(messages, list):
            for message in messages:
                process_message(message, year_counts, day_counts)
        elif isinstance(messages, dict) and 'messages' in messages:
            for message in messages['messages']:
                process_message(message, year_counts, day_counts)
        
        return year_counts, day_counts
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return defaultdict(int), defaultdict(int)

def process_message(message, year_counts, day_counts):
    timestamp = message.get('Timestamp', '')
    try:
        date = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        year_counts[str(date.year)] += 1
        day_counts[date.date()] += 1
    except ValueError:
        pass  # Skip messages with invalid timestamps

def count_messages(package_path):
    message_count = 0
    channel_counts = {}
    year_counts = defaultdict(int)
    day_counts = defaultdict(int)

    # Load the index.json file
    index_data = safe_read_json(os.path.join(package_path, 'index.json'))
    
    # Create a mapping of channel IDs to names
    channel_map = {str(k): v for k, v in index_data.items()}

    # Recursively search for message files
    for root, dirs, files in os.walk(package_path):
        for file in files:
            if file.endswith('.json') and file != 'index.json':
                file_path = os.path.join(root, file)
                file_year_counts, file_day_counts = count_messages_in_file(file_path)
                
                for year, count in file_year_counts.items():
                    year_counts[year] += count
                    message_count += count

                for day, count in file_day_counts.items():
                    day_counts[day] += count

                # Determine channel name
                relative_path = os.path.relpath(root, package_path)
                channel_id = relative_path.split(os.path.sep)[0]
                if channel_id.startswith('c'):
                    channel_id = channel_id[1:]  # Remove the 'c' prefix
                channel_name = channel_map.get(channel_id, f'Channel {channel_id}')
                
                channel_counts[channel_name] = channel_counts.get(channel_name, 0) + sum(file_year_counts.values())

    return message_count, channel_counts, year_counts, day_counts

def calculate_avg_messages_per_day(year_counts):
    avg_messages_per_day = {}
    for year, count in year_counts.items():
        year = int(year)
        days_in_year = 366 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 365
        avg_messages_per_day[year] = count / days_in_year
    return avg_messages_per_day

if __name__ == "__main__":
    package_path = "./Downloads/package/messages"
    total_messages, channel_counts, year_counts, day_counts = count_messages(package_path)
    avg_messages_per_day = calculate_avg_messages_per_day(year_counts)

    print("\n" + "="*50)
    print("Total Messages Sent:")
    print("="*50)
    print(total_messages)  # Only print the number

    print("\nMessages per Channel (sorted by count):")
    print("="*50)
    for channel, count in sorted(channel_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{channel}: {count}")

    print("\nMessages per Year and Average per Day:")
    print("="*50)
    for year in sorted(year_counts.keys()):
        count = year_counts[year]
        avg_per_day = avg_messages_per_day[int(year)]
        leap_year_status = "Leap Year" if int(year) % 4 == 0 and (int(year) % 100 != 0 or int(year) % 400 == 0) else "Non-Leap Year"
        print(f"{year} ({leap_year_status}): {count} messages, {avg_per_day:.2f} avg messages/day")

    print("\nTop 10 Days with Most Messages:")
    print("="*50)
    top_10_days = sorted(day_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    for date, count in top_10_days:
        print(f"{date.strftime('%Y-%m-%d')}: {count} messages")

In [ ]:
import matplotlib.pyplot as plt

# Function to clean channel names
def clean_channel_name(channel_name):
    if "Direct Message with" in channel_name:
        return channel_name.split("Direct Message with", 1)[1].strip()
    return channel_name.split(' - ')[-1] if ' - ' in channel_name else channel_name

# Sort the channels by message count
sorted_channels = sorted(channel_counts.items(), key=lambda x: x[1], reverse=True)

# Get the top 10 channels
top_10 = sorted_channels[:10]

# Calculate the sum of messages in other channels
other_count = sum(count for _, count in sorted_channels[10:])

# Prepare data for the pie chart
labels = [clean_channel_name(channel) for channel, _ in top_10] + [f"Other ({len(sorted_channels) - 10} channels)"]
sizes = [count for _, count in top_10] + [other_count]

# Discord-inspired color palette (updated with one more color)
discord_colors = [
    '#7289DA',  # Discord Blurple
    '#99AAB5',  # Discord Greyple
    '#2C2F33',  # Discord Dark
    '#23272A',  # Discord Not Quite Black
    '#FFB900',  # Discord Yellow
    '#5865F2',  # Discord New Blurple
    '#57F287',  # Discord Green
    '#ED4245',  # Discord Red
    '#FFA500',  # Orange 
    '#A84300',  # Darker Orange
    '#36393F',  # Discord Background Dark
]

# Create the pie chart
plt.figure(figsize=(12, 8))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=discord_colors,
)

# Change percentage text color to white for better visibility
for autotext in autotexts:
    autotext.set_color('white')  # Change color of percentage text to white

plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title(f"Top 10 Discord Channels by Message Count (Total: {total_messages:,} messages)", fontsize=16)



plt.tight_layout()
plt.show()